In [41]:
!pip install requests tqdm scikit-learn


In [42]:
import pandas as pd
import requests
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler


In [43]:
wallets = pd.read_csv('https://docs.google.com/spreadsheets/d/1ZzaeMgNYnxvriYYpe8PE7uMEblTI0GV5GIVUnsP-sBs/export?format=csv')
wallets.head()


,wallet_id
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0x0795732aacc448030ef374374eaae57d2965c16c
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


In [44]:
api_key = "RG9CAA7GM1SXWKH6ATHTYR5J58GS6P5IXC"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [45]:
def get_txns(wallet):
    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={wallet}&startblock=0&endblock=99999999&sort=asc&apikey={api_key}"
    r = requests.get(url)
    data = r.json()

    if data['status'] == '1':
        return data['result']
    else:
        return []  # Return empty list if no transactions or error


In [46]:
def extract_features(txns):
    total_supply = 0
    total_borrow = 0
    total_repay = 0
    active_borrow_positions = 0

    if not txns:
        return {
            'total_supplied': 0,
            'total_borrowed': 0,
            'repayment_ratio': 0,
            'active_borrow_positions': 0
        }

    for txn in txns:
        method_id = txn['input'][:10]

        # Placeholder logic for demonstration
        if method_id == "0xa9059cbb":  # ERC20 transfer, treat as supply (for simplicity)
            total_supply += float(txn['value']) / (10 ** 18)
        elif method_id == "0xborrowid":  # Placeholder borrow method
            total_borrow += float(txn['value']) / (10 ** 18)
        elif method_id == "0xrepayid":  # Placeholder repay method
            total_repay += float(txn['value']) / (10 ** 18)

    repayment_ratio = 1 if total_borrow == 0 else min(1, total_repay / total_borrow)
    active_borrow_positions = 1 if total_borrow > 0 else 0

    features = {
        'total_supplied': total_supply,
        'total_borrowed': total_borrow,
        'repayment_ratio': repayment_ratio,
        'active_borrow_positions': active_borrow_positions
    }

    return features


In [47]:
all_features = []

for wallet in tqdm(wallets['wallet_id']):
    txns = get_txns(wallet)
    features = extract_features(txns)
    features['wallet_id'] = wallet
    all_features.append(features)

feature_df = pd.DataFrame(all_features)
feature_df.head()


100%|██████████| 103/103 [00:23<00:00,  4.33it/s]


,total_supplied,total_borrowed,repayment_ratio,active_borrow_positions,wallet_id
0,0.0,0,1,0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0.0,0,1,0,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0.0,0,1,0,0x0795732aacc448030ef374374eaae57d2965c16c
3,0.0,0,1,0,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0.0,0,0,0,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


In [48]:
def normalize_features(feature_df):
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(feature_df)
    return pd.DataFrame(scaled, columns=feature_df.columns)


In [49]:
def compute_weighted_score(row):
    weights = {
        'total_supplied': 0.1,
        'total_borrowed': 0.15,
        'repayment_ratio': 0.25,
        'active_borrow_positions': 0.1
    }

    score = 0
    for feature, weight in weights.items():
        score += row[feature] * weight

    return int(score * 1000)


# New Section

In [50]:
# Normalize features except wallet_id
normalized_df = normalize_features(feature_df.drop(['wallet_id'], axis=1))

# Add wallet_id back
normalized_df['wallet_id'] = feature_df['wallet_id']

# Compute scores
normalized_df['score'] = normalized_df.apply(compute_weighted_score, axis=1)

# Select final columns
final_df = normalized_df[['wallet_id', 'score']]
final_df.head()


,wallet_id,score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,250
1,0x06b51c6882b27cb05e712185531c1f74996dd988,250
2,0x0795732aacc448030ef374374eaae57d2965c16c,250
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,250
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,0


In [51]:
final_df.to_csv('wallet_risk_scores.csv', index=False)
print("✅ CSV saved as wallet_risk_scores.csv")


✅ CSV saved as wallet_risk_scores.csv
